# qtl处理

In [ ]:
import csv
import os
import numpy as np
import pandas as pd

file_path = "/home/wcy/data/Tool_Data/smr/eye_qtl/"

for i in range(1,23):
    qtl_file = file_path + "new_nominal.eQTL.chr" + str(i) + ".with_thresholds.txt"
    save_path = file_path + "chr" + str(i) + ".txt"
    
    qtl = pd.read_csv(qtl_file,"\t")
    qtl = qtl.drop(['Effect_allele','Baseline_allele','genelevel_threshold','V14'], axis=1)
    qtl['is_signif'] = qtl['is_signif'].astype(str).apply(lambda x: '1' if x.upper() == 'TRUE' else ('0' if x.upper() == 'FALSE' else x))
    qtl = qtl[qtl['V8'].str.startswith('rs')]
    col = qtl['V8'].str.split(':').str[0]
    qtl['V8'] = col

    qtl.to_csv(save_path,'\t',index=False,header=False)


#print(save_path)

# qtl更新

In [16]:
import csv
import os
import numpy as np
import pandas as pd

esi_path = "/home/wcy/data/Tool_Data/smr/eye_qtl/eye_qtl.esi"
bim_path = "/home/wcy/data/UKB/test_data/eye_gwas/gene/eye_gene_hg38.bim"

esi = pd.read_csv(esi_path,'\t',header=None)
bim = pd.read_csv(bim_path,'\t',header=None)

esi = esi.drop([4,5,6], axis=1)
bim = bim.drop([2,3], axis=1)

esi_after = pd.merge(esi,bim,on=[1])
esi_after = esi_after.drop(['0_y'], axis=1)

esi_after.to_csv("/home/wcy/data/Tool_Data/smr/eye_qtl/eye_qtl_new.esi",'\t',index=False,header=False)

#print(esi_after)

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


# eye_eqtl处理

In [15]:
import csv
import os
import numpy as np
import pandas as pd

eye_qtl_path = "/home/wcy/data/Tool_Data/smr/eye_qtl/eye_qtl.txt"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/"
loci_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GenomicRiskLoci.txt"

eye_qtl = pd.read_csv(eye_qtl_path,'\t',header=None)
eye_qtl = eye_qtl.drop(columns=[1,2,3,4,5,6,8,9,10,13])
eye_qtl.rename(columns={0: 'gene_id',7: 'SNP',11: 'gene_p',12: 'gene_slope'}, inplace=True)

gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf', 'rsID': 'SNP'}, inplace=True)

loci = pd.read_csv(loci_path,'\t')
target_snps = loci['rsID']
for  target_snp in target_snps:
    target_snp = target_snp
    target_row = gwas[gwas['SNP'] == target_snp]
    traget_chr = target_row['CHR'].values[0]
    target_bp = target_row['POS'].values[0]
    chr_condition = gwas['CHR'] == traget_chr
    pos_condition = (gwas['POS'] >= target_bp - 500000) & (gwas['POS'] <= target_bp + 500000)

    result = gwas[chr_condition & pos_condition]
    merge_data = pd.merge(result,eye_qtl,on="SNP")
    merge_lead = merge_data.sort_values('gene_p').drop_duplicates(subset='SNP', keep='first')

    path = save_path + target_snp + ".txt"
    merge_lead.to_csv(path,'\t',index=False,header=True)
    print(path)
   

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs73045306.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs7422793.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs17421627.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs225726.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs2519093.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs17476364.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs5442.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs11045245.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs77542847.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs8070929.txt
/home/wcy/data/UKB/test_data/eye_gwas/coloc/0720/rs479486.txt
              SNP  CHR       POS EA NEA       maf      BETA        SE  \
91478    rs749776   19  49015801  T   C  0.422699 -0.021623  0.042071   
92354   rs3752210   19  49017115  A   T  0.417275 -0.020413  0.041966   
91405    rs753308   19  49015318  A   G  0.417638 -0.023004  0.041951   
92062   rs40024

# gtex-cloc处理

In [48]:
import csv
import os
import numpy as np
import pandas as pd

eqtl_path = "/home/wcy/data/Tool_Data/smr/eqtls/"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/"
loci_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GenomicRiskLoci.txt"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/file_test.txt"


gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf_snp', 'rsID': 'SNP'}, inplace=True)
gwas['variant_id'] = gwas.apply(lambda row: f"chr{row['CHR']}_{row['POS']}_{row['NEA']}_{row['EA']}_b38", axis=1)

loci = pd.read_csv(loci_path,'\t')
file_list = pd.read_csv(file_list,header=None)
target_snps = loci['rsID']

for folder_name in file_list[0]:
    result_folder = save_path + folder_name + "/"
    eqtl_file = eqtl_path + folder_name + ".v8.EUR.signif_pairs.txt"
    os.makedirs(result_folder, exist_ok=True)

    eqtl = pd.read_csv(eqtl_file,'\t')
    eqtl = eqtl.drop(columns=["tss_distance", "pval_nominal_threshold", "min_pval_nominal", "pval_beta"])
    eqtl.rename(columns={"phenotype_id": 'gene_id',"pval_nominal": 'p_gene',"slope": 'gene_slope',"slope_se": 'gene_slope_se'}, inplace=True)
    
    #print(result_folder,eqtl_file)

    for  target_snp in target_snps:
        target_row = gwas[gwas['SNP'] == target_snp]
        traget_chr = target_row['CHR'].values[0]
        target_bp = target_row['POS'].values[0]
        chr_condition = gwas['CHR'] == traget_chr
        pos_condition = (gwas['POS'] >= target_bp - 500000) & (gwas['POS'] <= target_bp + 500000)

        result = gwas[chr_condition & pos_condition]
        merge_data = pd.merge(result,eqtl,on="variant_id")
        merge_lead = merge_data.sort_values('p_gene').drop_duplicates(subset='SNP', keep='first')

        result_path = result_folder + target_snp + ".txt"
        merge_lead.to_csv(result_path,'\t',index=False,header=True)


/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except 

# gtex-smr处理

In [2]:
import csv
import os
import numpy as np
import pandas as pd

eqtl_path = "/home/wcy/data/Tool_Data/smr/eQTL_besd_lite/"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/C-GWAS_hg19.ma"
bfile_path = "/home/wcy/data/UKB/test_data/eye_gwas/gene/eye_gene"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/file_list.txt"


file_list = pd.read_csv(file_list,header=None)


for file_name in file_list[0]:
    eqtl_file = eqtl_path + file_name + ".lite"
    save_file = save_path + file_name
    
    cmd = f"~/python_code/smr_linux_x86_64 --bfile {bfile_path} --gwas-summary {gwas_path} --beqtl-summary {eqtl_file} --out {save_file} --thread-num 70"
    os.system(cmd)

/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Adipose_Subcutaneous
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Adipose_Visceral_Omentum
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Adrenal_Gland
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Artery_Aorta
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Artery_Coronary
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Artery_Tibial
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Amygdala
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Anterior_cingulate_cortex_BA24
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Caudate_basal_ganglia
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Cerebellar_Hemisphere
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Cerebellum
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Cortex
/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/Brain_Frontal_Cortex_BA9
/home/wcy/data/UKB/test_data/eye

# gtex-cloc处理 (新)

In [10]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

eqtl_path = "/home/wcy/data/Tool_Data/smr/test/"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/"
loci_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GenomicRiskLoci.txt"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/file_list.txt"

selected_columns = ['variant', 'rsid', 'pvalue', 'maf', 'gene_id']
#
#
gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf_snp', 'rsID': 'rsid'}, inplace=True)
gwas['variant_id'] = gwas.apply(lambda row: f"chr{row['CHR']}_{row['POS']}_{row['NEA']}_{row['EA']}_b38", axis=1)

loci = pd.read_csv(loci_path,'\t')
file_list = pd.read_csv(file_list,header=None)
target_snps = loci['rsID']

for folder_name in file_list[0]:
    result_folder = save_path + folder_name + "/"
    eqtl_file = eqtl_path + folder_name + ".tsv"
    os.makedirs(result_folder, exist_ok=True)

    eqtl = pd.read_csv(eqtl_path,'\t',usecols=selected_columns)
   


    for  target_snp in target_snps:
        target_row = gwas[gwas['rsid'] == target_snp]
        traget_chr = target_row['CHR'].values[0]
        target_bp = target_row['POS'].values[0]
        chr_condition = gwas['CHR'] == traget_chr
        pos_condition = (gwas['POS'] >= target_bp - 500000) & (gwas['POS'] <= target_bp + 500000)

        result = gwas[chr_condition & pos_condition]
        merge_data = pd.merge(result,eqtl,on="rsid")
        merge_lead = merge_data.sort_values('pvalue').drop_duplicates(subset='rsid', keep='first')

        result_path = result_folder + target_snp + ".txt"
        merge_lead.to_csv(result_path,'\t',index=False,header=True)


               rsid  CHR       POS EA NEA   maf_snp      BETA        SE  \
20614     rs2292112   19  48296459  A   G  0.310311 -0.045096  0.045027   
136549    rs4002462   19  49016740  A   G  0.429996 -0.022243  0.041997   
32750    rs12608580   19  48356974  A   G  0.312908 -0.043095  0.044820   
135858     rs749776   19  49015801  T   C  0.422699 -0.021623  0.042071   
78972    rs12979144   19  48734290  A   G  0.420140  0.037437  0.041893   
...             ...  ...       ... ..  ..       ...       ...       ...   
52408   rs114600583   19  48569948  A   G  0.138805 -0.093561  0.059478   
50329    rs80063723   19  48568852  A   C  0.138824 -0.093591  0.059467   
51022   rs184033255   19  48569264  C   T  0.138805 -0.093561  0.059478   
56170     rs7249118   19  48575582  A   G  0.329583 -0.079026  0.043950   
183522   rs62127956   19  49225061  A   G  0.240746  0.027497  0.048050   

               snp_p      N              variant_id             variant  \
20614   4.673796e-01  26

In [1]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

input_path = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/"
output_path = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/adjust/"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/file_list.txt"

file_list = pd.read_csv(file_list,header=None)
for folder_name in file_list[0]:
    input_file = input_path + folder_name + ".smr"
    output_file = output_path + folder_name + ".csv"
    
    cmd = f"/usr/bin/Rscript  ~/data/UKB/test_data/eye_gwas/smr_adjust.r -i {input_file} -o {output_file}"
    os.system(cmd)



Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading required package: stringr
Loading requir

In [1]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

input_path = "/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/adjust/"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GTex/file_list.txt"

result_df = pd.DataFrame()
file_list = pd.read_csv(file_list,header=None)
for folder_name in file_list[0]:
    input_file = input_path + folder_name + ".csv"
    input = pd.read_csv(input_file, sep=',')

    if (input['p_adjust'] < 0.05).any():
        # 将满足条件的行及对应的folder_name添加到结果DataFrame中
        input['folder_name'] = folder_name
        result_df = result_df.append(input.loc[input['p_SMR'] < 8.4e-6])

result_df.to_csv("/home/wcy/data/UKB/test_data/eye_gwas/smr/0721/result/adjust/result_sig_new.csv","\t",header=True,index=False)

/tmp/ipykernel_1160926/1308696258.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(input.loc[input['p_SMR'] < 8.4e-6])
/tmp/ipykernel_1160926/1308696258.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(input.loc[input['p_SMR'] < 8.4e-6])
/tmp/ipykernel_1160926/1308696258.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(input.loc[input['p_SMR'] < 8.4e-6])
/tmp/ipykernel_1160926/1308696258.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(input.loc[input['p_SMR'] < 8.4e-6])
/tmp/ipykernel_1160926/1308696258.py

# 更新链信息

In [1]:
import requests
from bs4 import BeautifulSoup

def get_strand_symbol(strand):
    return "+" if strand == 1 else "-"

def get_gene_info(gene):
    url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene}?expand=1"
    response = requests.get(url)
    if response.ok:
        soup = BeautifulSoup(response.text, "html.parser")
        gene_id = soup.find("title").text.strip().split("::")[-1]
        pre_tag = soup.find("pre")
        gene_info = pre_tag.text.strip().split()
        gene_start = gene_info[-5]
        gene_end = gene_info[-19]
        gene_strand = gene_info[-3]
        gene_info_str = f"{gene}\t{gene_start}\t{gene_end}\t{get_strand_symbol(int(gene_strand))}"
        print(gene_info_str)
        return gene_info_str
    else:
        print(f"Error: {response.status_code}")
    return ""

# 读取基因列表文件，每行一个基因名
file_path = "/home/wcy/data/Tool_Data/smr/glist-hg19-0801"
output_file = "/home/wcy/data/Tool_Data/smr/glist-hg19-new-0801"

with open(file_path, "r") as file:
    gene_list = [line.strip().split()[3] for line in file]
   
# 查询每个基因的链信息并写入文件
with open(output_file, "w") as output:
    for gene in gene_list:
        gene_info = get_gene_info(gene)
        output.write(gene_info + "\n")

print("Gene information updated in the output file.")


TDRP	489803	545781	-
TEAD1	12674421	12944737	+
TEAD2	49340595	49362457	-
TEAD3	35473597	35497079	-
TEAD4	2959330	3040676	+
TEC	48135783	48269838	-
TECPR1	98214624	98252232	-
TECPR2	102362941	102502477	+
TECR	14517085	14565980	+
TECRL	64275257	64409468	-
TECTA	121101243	121191490	+
TECTB	112283400	112305038	+
TEDDM1	182398117	182400667	-
TEF	41367333	41399326	+
TEFM	30897336	30906238	-
TEK	27109141	27230174	+
TEKT1	6789133	6831761	-
TEKT2	36084094	36088275	+
TEKT3	15303811	15341632	-
TEKT4	94871430	94876823	+
TEKT4P2	9068414	9073609	-
TEKT5	10627501	10694930	-
TELO2	1493344	1510457	+
TEN1	75979240	76000586	+
TEN1-CDK3	75979231	76005999	+
Error: 400
TENM1	124375903	125204312	-
TENM2	166979029	168264157	+
TENM3	182143987	182803024	+
TENM4	78652829	79441030	-
TEP1	20365667	20413501	-
TEPP	57976435	57988116	+
TERC	169764520	169765060	-
TERF1	73008856	73048123	+
TERF2	69355567	69408571	-
TERF2IP	75647773	75761872	+
TERT	1253147	1295068	-
TES	116210506	116258783	+
TESC	117038923	117099479	-
T

# 合并csv

In [13]:
import os
import pandas as pd
from tqdm import tqdm

# 指定包含CSV文件的文件夹路径
folder_path = '/home/wcy/data/Tool_Data/smr/eye_qtl_new/result'
save_path = '/home/wcy/data/Tool_Data/smr/eye_qtl_new/'

# 获取文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 创建一个空的DataFrame来存储合并后的数据
merged_data = pd.DataFrame()

# 循环遍历每个CSV文件并合并到DataFrame中
for file in tqdm(csv_files):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, header=0)  # 假设表头在第一行
    merged_data = pd.concat([merged_data, df], axis=0)

# 将合并后的数据保存为一个新的CSV文件
merged_file_path = os.path.join(folder_path, 'merged_data.csv')
merged_data.to_csv(merged_file_path, index=False)

print(f"Merged data saved to {merged_file_path}")



100%|██████████| 18701/18701 [24:08<00:00, 12.91it/s]


Merged data saved to /home/wcy/data/Tool_Data/smr/eye_qtl_new/result/merged_data.csv


# 爬虫数据转smr格式

In [1]:
import csv
import os
import numpy as np
import pandas as pd
# 读取CSV文件
csv_file_path = "/home/wcy/data/Tool_Data/smr/eye_qtl_new/eqtl_raw.csv"  # 替换为实际的文件路径
save_dir = "/home/wcy/data/Tool_Data/smr/eye_qtl_new/step1/" 
data = pd.read_csv(csv_file_path)


## 根据tissue提取
Retina_non = data[data['tissue'] == "Retina, non-Macula"]
Retina = data[data['tissue'] == "Retina, Macula"]
RPE_non = data[data['tissue'] == "RPE, non-Macula"]
RPE = data[data['tissue'] == "RPE, Macula"]


name_list = {'Retina_non':Retina_non,'Retina':Retina, 'RPE_non':RPE_non, 'RPE':RPE}
for name_str,name in name_list.items():
    name_result = name.loc[:, ['snps','ensemble','beta','statistic','pvalue','fdr']]
    name_result.rename(columns={'snps':'SNP','ensemble':'gene','statistic':'t-stat','pvalue':'p-value','fdr':'FDR',},inplace=True)
    name_list = name.loc[:, ['chromosome','snps','distance_snp_to_start','snp_bp','alt','ref','maf']]
    name_list['distance_snp_to_start'] = 0
    name_list['maf'] = "NA"
    name_gene = name.loc[:, ['chromosome','ensemble','snp_bp','gene_start','symbol','ref']]
    name_gene['snp_bp'] = 0
    name_gene['ref'] = "NA"
    name_gene = name_gene.drop_duplicates(subset="ensemble")

    file_name_result = save_dir +  f"{name_str}_result.csv"  # 修改这里
    file_name_list = save_dir +  f"{name_str}_list.csv"
    file_name_gene = save_dir +  f"{name_str}_gene.csv"

    name_result.to_csv(file_name_result,sep="\t",index=False)
    name_list.to_csv(file_name_list,sep="\t",index=False,header=False)
    name_gene.to_csv(file_name_gene,sep="\t",index=False,header=False)


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_strand_symbol(strand):
    return "+" if strand == 1 else "-"

def get_gene_info(gene):
    url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene}?expand=1"
    response = requests.get(url)
    if response.ok:
        soup = BeautifulSoup(response.text, "html.parser")
        gene_id = soup.find("title").text.strip().split("::")[-1]
        pre_tag = soup.find("pre")
        gene_info = pre_tag.text.strip().split()
        gene_strand = gene_info[-3]
        gene_info_str = f"{get_strand_symbol(int(gene_strand))}"
        print(gene_info_str)
        return gene_info_str
    else:
        print(f"Error: {response.status_code}")
    return ""


def add_strand_info(row):
    gene_name = row[4]
    gene_info_str = get_gene_info(gene_name)
    return gene_info_str

# 读取基因列表文件，每行一个基因名
data = pd.read_csv("/home/wcy/data/Tool_Data/smr/eye_qtl_new/step1/RPE_non_gene.csv",sep='\t',header=None)

# 在数据表格中添加新的一列来存储链信息
data['gene_info'] = data.apply(add_strand_info, axis=1)

data.to_csv("/home/wcy/data/Tool_Data/smr/eye_qtl_new/step3/RPE_non_gene.csv",sep='\t',index=False)
# 打印带有链信息的数据表格
print(data)


-
-
+
Error: 400
+
-
+
+
-
Error: 400
Error: 400
+
-
Error: 400
Error: 400
Error: 400
+
+
-
Error: 400
-
Error: 400
-
Error: 400
-
+
Error: 400
Error: 400
-
Error: 400
+
Error: 400
Error: 400
-
+
Error: 400
-
Error: 400
-
-
+
-
-
-
+
+
-
-
-
Error: 400
+
-
+
-
Error: 400
-
Error: 400
-
-
Error: 400
-
-
Error: 400
+
Error: 400
+
-
+
+
-
+
-
+
Error: 400
+
Error: 400
+
-
+
Error: 400
Error: 400
-
+
-
-
Error: 400
Error: 400
-
-
Error: 400
-
+
+
+
-
-
Error: 400
-
+
+
-
Error: 400
Error: 400
Error: 400
+
Error: 400
-
+
-
-
-
+
Error: 400
-
-
-
+
-
Error: 400
-
-
+
-
+
+
-
Error: 400
+
+
-
-
+
Error: 400
+
+
-
-
-
-
Error: 400
+
-
Error: 400
+
Error: 400
Error: 400
+
+
Error: 400
Error: 400
+
+
-
+
+
-
-
+
Error: 400
+
Error: 400
+
+
-
Error: 400
-
+
+
-
-
Error: 400
+
-
+
Error: 400
Error: 400
+
+
+
+
Error: 400
Error: 400
Error: 400
+
-
-
+
+
Error: 400
-
+
-
Error: 400
-
-
Error: 400
+
Error: 400
+
-
+
+
-
-
-
Error: 400
Error: 400
-
+
Error: 400
+
Error: 400
-
+
Error: 400
+
-
-
Error:

# 爬虫转coloc

In [4]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

eqtl_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/result/"
loci_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GenomicRiskLoci.txt"
file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/file.txt"

selected_columns = ['snps', 'pvalue', 'maf', 'ensemble']
#
#
gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf_snp', 'rsID': 'snps'}, inplace=True)
gwas['variant_id'] = gwas.apply(lambda row: f"chr{row['CHR']}_{row['POS']}_{row['NEA']}_{row['EA']}_b38", axis=1)

loci = pd.read_csv(loci_path,'\t')
file_list = pd.read_csv(file_list,header=None)
target_snps = loci['rsID']

for folder_name in file_list[0]:
    result_folder = save_path + folder_name + "/"
    eqtl_file = eqtl_path + folder_name + ".csv"
    os.makedirs(result_folder, exist_ok=True)

    eqtl = pd.read_csv(eqtl_file,'\t',usecols=selected_columns)
   


    for  target_snp in target_snps:
        target_row = gwas[gwas['snps'] == target_snp]
        traget_chr = target_row['CHR'].values[0]
        target_bp = target_row['POS'].values[0]
        chr_condition = gwas['CHR'] == traget_chr
        pos_condition = (gwas['POS'] >= target_bp - 500000) & (gwas['POS'] <= target_bp + 500000)

        result = gwas[chr_condition & pos_condition]
        merge_data = pd.merge(result,eqtl,on="snps")
        merge_lead = merge_data.sort_values('pvalue').drop_duplicates(subset='snps', keep='first')

        result_path = result_folder + target_snp + ".txt"
        merge_lead.to_csv(result_path,'\t',index=False,header=True)


/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except 

# eQTL_gene转coloc

In [1]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

eqtl_path = "/home/wcy/data/Tool_Data/smr/eQTL_gene/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eqtl_gene/"
loci_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/GenomicRiskLoci.txt"
#file_list = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/file.txt"

selected_columns = ['SNP', 'Pvalue', 'Gene', 'GeneSymbol']
#
#
gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf_snp', 'rsID': 'SNP'}, inplace=True)
gwas['variant_id'] = gwas.apply(lambda row: f"chr{row['CHR']}_{row['POS']}_{row['NEA']}_{row['EA']}_b38", axis=1)

loci = pd.read_csv(loci_path,'\t')
target_snps = loci['rsID']


eqtl = pd.read_csv(eqtl_path,'\t',usecols=selected_columns)

for  target_snp in target_snps:
    target_row = gwas[gwas['SNP'] == target_snp]
    traget_chr = target_row['CHR'].values[0]
    target_bp = target_row['POS'].values[0]
    chr_condition = gwas['CHR'] == traget_chr
    pos_condition = (gwas['POS'] >= target_bp - 500000) & (gwas['POS'] <= target_bp + 500000)
    result = gwas[chr_condition & pos_condition]
    merge_data = pd.merge(result,eqtl,on="SNP")
    merge_lead = merge_data.sort_values('Pvalue').drop_duplicates(subset='SNP', keep='first')
    result_path = save_path + target_snp + ".txt"
    merge_lead.to_csv(result_path,'\t',index=False,header=True)

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
import csv
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

eqtl_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/"
gwas_path = "/home/wcy/data/UKB/test_data/eye_gwas/cgwas/result_0720/C-GWAS_hg38.txt"
save_path = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/eye_qtl/result/"
mqtl_file = "/home/wcy/data/UKB/test_data/eye_gwas/coloc/mqtl/rs2519093.csv"

selected_columns = ['snps', 'pvalue', 'maf', 'ensemble']
#
#
gwas = pd.read_csv(gwas_path,'\t')
gwas = gwas.drop(columns=['STATUS'])
gwas.rename(columns={'P': 'snp_p','EAF': 'maf_snp', 'rsID': 'rsid'}, inplace=True)
gwas['variant_id'] = gwas.apply(lambda row: f"chr{row['CHR']}_{row['POS']}_{row['NEA']}_{row['EA']}_b38", axis=1)

#eqtl = pd.read_csv(eqtl_file,'\t',usecols=selected_columns)

mqtl = pd.read_csv(mqtl_file,',')
merge_data = pd.merge(gwas,mqtl,on="rsid")
merge_lead = merge_data.sort_values('pval').drop_duplicates(subset='rsid', keep='first')
#print(merge_lead)
merge_lead.to_csv("/home/wcy/data/UKB/test_data/eye_gwas/coloc/mqtl/rs2519093_snp_mqtl.txt",sep="\t",index=False,header=True)



/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
import pandas as pd

glist_old = pd.read_csv("/home/wcy/data/Tool_Data/smr/glist/glist-hg19-old",sep=" ")
glist_new = pd.read_csv("/home/wcy/data/Tool_Data/smr/glist/glist-hg19-merge",sep="\t")
glist_new = glist_new.drop(columns=['strat','end'])
merge = pd.merge(glist_old,glist_new,on="gene")
merge.to_csv("/home/wcy/data/Tool_Data/smr/glist/glist-hg19-merge-new",sep=" ",index=False,header=False)
print(merge)

      chr      strat        end      gene stran
0      19   58858171   58864865      A1BG     -
1      19   58863335   58866549  A1BG-AS1     +
2      10   52559168   52645435      A1CF     -
3      12    9220303    9268558       A2M     -
4      12    9217772    9220651   A2M-AS1     +
...    ..        ...        ...       ...   ...
26051   1   53308182   53360247    ZYG11A     +
26052   1   53192130   53293013    ZYG11B     +
26053   7  143078359  143088206       ZYX     +
26054  17    3907738    4046253     ZZEF1     -
26055   1   78030189   78148343      ZZZ3     -

[26056 rows x 5 columns]
